<a href="https://colab.research.google.com/github/Viniciusp67/Consultor-de-Direito-Eleitoral/blob/main/dir_eleitoral_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#https://huggingface.co/
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [5]:
PATH = './sample_data/'
dados_treino = 'codigo_eleitoral.txt'

In [7]:
from tokenizers import ByteLevelBPETokenizer

# Initialize a ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=[PATH+dados_treino], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [8]:
tokenizer.encode("Direito eleitoral simplificado").ids

[40, 77, 267, 760, 410, 770, 655, 311, 3165]

In [9]:
tokenizer.decode([40, 77, 267, 760, 410, 770, 655, 311, 3165])

'Direito eleitoral simplificado'

In [10]:
!rm -r ./sample_data/RAW_MODEL
!mkdir ./sample_data/RAW_MODEL
tokenizer.save_model(PATH+'RAW_MODEL')

rm: cannot remove './sample_data/RAW_MODEL': No such file or directory


['./sample_data/RAW_MODEL/vocab.json', './sample_data/RAW_MODEL/merges.txt']

In [11]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL', max_len=512)

In [12]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [13]:
model.num_parameters()

83502880

In [14]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=PATH+dados_treino,
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [16]:
dataset.examples[:3]

[{'input_ids': tensor([   0,   39,  660,  516, 4319,  430,  312,  419,   18,   27, 2087,   16,
           265,  664,  265, 4606,  265, 3136,   25,    2])},
 {'input_ids': tensor([   0, 4242, 5218, 3876, 1362,  263,   17,  372, 2091, 4242, 4391, 1362,
          2751,  407, 1851,   83,   79, 4242, 4391, 1362, 2312, 4242, 4391, 1362,
           225,   59,  434,   88,   87,   37,   84,   84,    2])},
 {'input_ids': tensor([   0,   45, 1170,   77,  278, 1190,  516,   18,    2])}]

In [18]:
tokenizer.decode(dataset.examples[8]['input_ids'])

'<s>Parágrafo único. O Tribunal Superior Eleitoral expedirá instruções para sua fiel execução.</s>'